<a href="https://colab.research.google.com/github/ajkerrigan/my-itchy-dog/blob/main/Ginger_Scratching_Trends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install --upgrade altair pyarrow

In [2]:

import requests
from google.colab import userdata

auth_token = requests.post(
    'https://app.whistle.com/api/login',
    json={
        'email': userdata.get('email'),
        'refresh_token': userdata.get('refresh_token')
    }
).json()['auth_token']

scratching = requests.get(
    'https://app.whistle.com/api/pets/511006/health/graphs/scratching?num_of_days=180',
    headers={
        'Accept': "application/vnd.whistle.com.v4+json",
        'Authorization': f'Bearer {auth_token}'
    }
).json()['data']


In [9]:

import polars as pl

df = pl.DataFrame(scratching)

In [33]:
import altair as alt

scale = alt.Scale(domain=['infrequent', 'occasional', 'elevated', 'severe'], range=['green', 'yellow', 'orange', 'red'])
alt.Chart(df, title="Ginger's Scratching").mark_bar().encode(
    x=alt.X('start_date', type='temporal').title('Date'),
    y=alt.Y('score', type='quantitative').title('Scratching Time (seconds)'),
    color=alt.Color('status', type='nominal', scale=scale)
).properties(
    width='container',
    height=600,
)

alt.Chart(...)